# Outline
0. Discrete space setup
1. Discrete Ergodic Coverage
2. Maximizing Information Gathering

In [22]:
distribution_name = "close_peaks"
time_steps = 100
K = 5
system1_name = "casadi"
system2_name = "ig"
compute_mu = False
grid_length = 10

dx = 0.2

In [16]:
import numpy as np
U_shape = (1,1)
all_k = list(np.ndindex(*[K]*len(U_shape)))

# 0. Discrete Space Setup

## 0.a. Define probability distribution $\mu$

In [3]:
# defining probability distribution mu
from probability_distribution import *
mu_unnorm = mu_gaussians([(np.array([0.25, 0.75]), 0.025), (np.array([0.5, 0.5]), 0.025)], U_shape)
# mu_unnorm = mu_gaussians([(np.array([0.25, 0.75]), 0.025), (np.array([0.5, 0.5]), 0.025)], U_shape)
# mu_unnorm = mu_gaussians([(np.array([0.25, 0.75]), 0.01), (np.array([0.6, 0.3]), 0.05)], U_shape)
# mu_unnorm = mu_gaussians([(np.array([0.25, 0.75]), 0.05), (np.array([0.6, 0.3]), 0.1)], U_shape)
total = mu_total(mu_unnorm, U_shape)
# mu = mu_gaussians([(np.array([0.2, 0.75]), 0.1)], U_shape)
mu = lambda x: mu_unnorm(x)/total # mu_normalize(mu_unnorm, U_shape)

# calculating fourier coefficients of probability distribution mu
from fourier_functions import *
import pickle

ff = Fourier_Functions(mu, U_shape, K, compute_mu=compute_mu)
if compute_mu:
    mu_k = {}
    for k in ff:
        mu_k[k] = ff[k]['mu_k']

    with open(f'mu/{distribution_name}_{K}.pkl', 'wb') as handle:
        pickle.dump(mu_k, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f'mu/{distribution_name}_{K}.pkl', 'rb') as handle:
        mu_k = pickle.load(handle)
    for k in ff:
        ff[k]['mu_k'] = mu_k[k]

original_mu = mu 
mu = fourier_coefficient2distribution(ff, all_k, c_k=None)


## 1. Generate Vanilla Ergodic Trajectory $x$

In [4]:
from ergodic_agents import *
from mm_agent import *
from casadi_agent import *

agent1 = CasadiAgent(0, np.array([0.2, 0.3]), 0.5, all_k, U_shape, ff, eps=1e-5)
system1 = AgentSystem([agent1], mu, U_shape, ff, K)

In [5]:
t = 0
delta_t = 0.1
for i in range(time_steps):
    t = i * delta_t
    system1.evolve(t, delta_t)


(2, 100)
(2, 100)

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  86.38ms (  2.79ms)  86.04ms (  2.78ms)        31
       nlp_g  |   2.10ms ( 67.74us)   2.08ms ( 67.20us)        31
  nlp_grad_f  | 154.45ms (  4.83ms) 153.90ms (  4.81ms)        32
  nlp_hess_l  |  67.41 s (  2.25 s)  67.21 s (  2.24 s)        30
   nlp_jac_g  |  26.83ms (838.56us)  26.78ms (836.79us)        32
       total  |  68.45 s ( 68.45 s)  68.26 s ( 68.26 s)         1


In [6]:
filename1 = f"optimality/system={system1_name}_mu={distribution_name}_K={K}_T={time_steps}"
description = f"{system1_name} on {distribution_name}"

system1.visualize_trajectory(filename1, description)
system1.visualize_ergodicity(filename1)

## 2. Maximize Information Gathering

In [17]:
_mu_casadi = mu_gaussians_casadi([(np.array([0.25, 0.75]), 0.025), (np.array([0.5, 0.5]), 0.025)], U_shape)
# _mu_casadi = mu_gaussians_casadi([(np.array([0.25, 0.75]), 0.01), (np.array([0.6, 0.3]), 0.05)], U_shape)
# _mu_casadi = mu_gaussians_casadi([(np.array([0.2, 0.75]), 0.1)], U_shape)
mu_casadi = lambda x: _mu_casadi(x)/total

In [23]:
from casadi_agent_info_gathering import *
agent2 = CasadiAgentInfoGathering(0, np.array([0.2, 0.3]), 0.5, all_k, U_shape, ff, eps=1e-5)
agent2.mu = mu_casadi #lambda x: casadi.norm_2(x) # need to do gaussian but for casadi
agent2.dx = dx
system2 = AgentSystem([agent2], mu_casadi, U_shape, ff, K)

In [24]:
print(agent2.x_log[0].shape)

(2,)


In [25]:
t = 0
delta_t = 0.1
for i in range(time_steps):
    t = i * delta_t
    system2.evolve(t, delta_t)

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 601.98ms (  8.36ms) 602.39ms (  8.37ms)        72
       nlp_g  |   9.53ms (132.39us)   9.48ms (131.65us)        72
  nlp_grad_f  |   1.40 s ( 19.14ms)   1.40 s ( 19.15ms)        73
  nlp_hess_l  | 133.03 s (  1.87 s) 133.03 s (  1.87 s)        71
   nlp_jac_g  |  97.96ms (  1.34ms)  98.29ms (  1.35ms)        73
       total  | 137.31 s (137.31 s) 137.31 s (137.31 s)         1


In [26]:
filename2 = f"optimality/system={system2_name}_mu={distribution_name}_dx={dx}_T={time_steps}"
description = f"{system2_name} on {distribution_name}"

system2.visualize_trajectory(filename2, description)
# system2.visualize2d(filename=filename2, additional_title=description, plot_c_k=False)
system2.visualize_ergodicity(filename2)

## 3. Comparison

In [17]:
results = {}

results['ergodic_trajectory'] = {}
results['ergodic_trajectory']['trajectory'] = np.array(agent1.x_log)
results['ergodic_trajectory']['ergodicity'] = agent1.e_log
results['ergodic_trajectory']['info gathered'] = [info_gathering_metric_noncasadi(mu, agent1.x_log, T+1, 1/dx) for T in range(time_steps)]

results['info_gather_trajectory'] = {}
results['info_gather_trajectory']['trajectory'] = np.array(agent2.x_log)
results['info_gather_trajectory']['ergodicity'] = np.array(agent2.e_log)
results['info_gather_trajectory']['info gathered'] = np.array([info_gathering_metric(mu_casadi, agent2.x_log, T+1, 1/dx, T+1) for T in range(time_steps)]).reshape(-1,)


In [18]:
filename = f"optimality/system={system1_name}_{system2_name}_mu={distribution_name}_K={K}_dx={dx}_T={time_steps}"

with open(f'{filename}_results.pkl', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

def plot_trajectory_values(results, filename, trajectory_name):
    plt.figure()
    plt.plot(results[trajectory_name]['ergodicity'][1:], results[trajectory_name]['info gathered'], 'o')
    plt.xlabel("Ergodicity")
    plt.xscale("log")
    plt.gca().invert_xaxis()
    plt.ylabel("Information Gathered")
    plt.savefig(f"{filename}_ergodicity_vs_info_gathered.pdf")
plot_trajectory_values(results, filename1, 'ergodic_trajectory')
plot_trajectory_values(results, filename2, 'info_gather_trajectory')

trajectories = [results['ergodic_trajectory']['trajectory'], results['info_gather_trajectory']['trajectory']]
visualize_trajectory(f"{filename}_ergodicity_vs_info_gathered", 
                     f"Ergodic vs Info Gathering Trajectory", 
                     U_shape, trajectories, mu)


In [19]:
for trajectory in ['ergodic_trajectory', 'info_gather_trajectory']:
    print("Trajectory: ", trajectory)
    print("Final ergodicity: ", results[trajectory]['ergodicity'][-1])
    print("Final Info Gather: ", results[trajectory]['info gathered'][-1])

Trajectory:  ergodic_trajectory
Final ergodicity:  1.939733715033913e-08
Final Info Gather:  3.6086141608097564
Trajectory:  info_gather_trajectory
Final ergodicity:  0.024780407724141654
Final Info Gather:  -8.171378168402923
